In [1]:
from sae_lens import HookedSAETransformer, SAE, SAEConfig
from gemma_utils import get_gemma_2_config, gemma_2_sae_loader, get_all_string_min_l0_resid_gemma
import numpy as np
import torch
import tqdm
import einops
import re
from jaxtyping import Int, Float
from typing import List, Optional, Any
from torch import Tensor
import json
import os
from torch.utils.data import Dataset, DataLoader
import random
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import random
from transformer_lens.utils import get_act_name

In [2]:
torch.cuda.empty_cache()
np.random.seed(0)
random.seed(0)
torch.random.manual_seed(0)

In [3]:
model = HookedSAETransformer.from_pretrained("google/gemma-2-2b-it", device = "cpu")
#model.to("cuda:0")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer
Moving model to device:  cuda:0


HookedSAETransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-25): 26 x TransformerBlock(
      (ln1): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln1_post): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2_post): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
      

In [4]:
from datasets import load_dataset

ds = load_dataset("santhoshmlops/Skai_Gemma_Instruct_ChatTemplate",split = "train")
text = ds[:1000]['text']
tokenized_ds = model.tokenizer(text, return_tensors = "pt", padding = True)
pairs_length = tokenized_ds['attention_mask'].sum(dim = -1).numpy()
indices = torch.where(torch.tensor(np.logical_and(pairs_length>50, pairs_length<200)))
text_array = np.array(text)[indices]
samp_ind = torch.randperm(len(text_array))[:300]
final_text = text_array[samp_ind].tolist()

In [5]:

text = final_text[0]
text = text.split("<statr_of_turn>model")[0]+"<start_of_turn>model"
tokens = model.to_tokens(text, prepend_bos = False)
with torch.no_grad():
        out = model.generate(
            tokens,
            max_new_tokens = 400,
            temperature = 0.7,
            top_p = 0.9,
            eos_token_id=107,
            stop_at_eos=True,
            )
        
print(out)

  0%|          | 0/400 [00:00<?, ?it/s]

tensor([[     2,    106,   1645,  30641,    603,    671,  14239,    674,  19306,
            476,   6911, 235265,  15615,    476,   3590,    674,  47258,  56382,
            573,   3853, 235265,    109,  47239,    919, 235248, 235284,  53190,
         235269, 235248, 235304,  35768,    578, 235248, 235284,  83699, 235265,
          17487,    919, 235248, 235318,  63495, 235269, 235248, 235274, 235284,
          41612,    578, 235248, 235284, 108160, 235265,  27161,    919, 235248,
         235274,   1226, 235269, 235248, 235274, 235276,  35768,    578, 235248,
         235318,  64322, 235265,   2250,   1767,   7091,   8398,    749,  47239,
         235269,  17487,    578,  27161,    791,    575,   3051, 235336, 235248,
            107,    108,    106,   2516, 235248,    108, 227128, 235269,  35768,
         235269,  64322,    578,  41612,    708,  17203,   5604,    577,    614,
           7091,   8398,   1861,    984,    708,  17203,   9244,    575,  26511,
            578,    708,  13

In [6]:

from attribution_utils import calculate_feature_attribution 
from torch.nn.functional import log_softmax

In [7]:

def metric_fn(logits: torch.Tensor) -> torch.Tensor:
    log_probs = log_softmax(logits, dim = -1)
    #x = - torch.gather(log_probs[:,:-1,:],-1, tokens[:,1:].unsqueeze(-1))
    x = log_probs[:,-1,107]
    return x

In [8]:

full_strings= get_all_string_min_l0_resid_gemma()
saes_dict = {}
with torch.no_grad():
    for layer in tqdm.tqdm([22,23]):

        repo_id = "google/gemma-scope-2b-pt-res"
        folder_name = full_strings[layer]

        config = get_gemma_2_config(repo_id, folder_name)
        cfg, state_dict, log_spar = gemma_2_sae_loader(repo_id, folder_name)
        sae_cfg = SAEConfig.from_dict(cfg)
        sae = SAE(sae_cfg)
        sae.load_state_dict(state_dict)
        #sae.to("cuda:0")
        saes_dict[sae.cfg.hook_name] = sae

/root/GemmaExploration/gemma_utils.py:158: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  resid_dict = df[df['release'] == "gemma-scope-2b-pt-res"]['saes_map'][0]
100%|██████████| 2/2 [00:01<00:00,  1.12it/s]


In [9]:

tokens = torch.tensor(out.tolist())
feature_attribution_df = calculate_feature_attribution(
    model = model,
    input = tokens,
    metric_fn = metric_fn,
    include_saes=saes_dict,
    include_error_term=True,
    return_logits=True,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 66.94 MiB is free. Process 711445 has 23.59 GiB memory in use. Of the allocated memory 22.83 GiB is allocated by PyTorch, and 463.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)